In [7]:
import pandas as pd

# Load files
import_df = pd.read_csv("../data/Processed/Import_Trade_Cleaned.csv")
export_df = pd.read_csv("../data/Processed/Export_Trade_Cleaned.csv")

# Check column names
print("Import Columns:", import_df.columns.tolist())
print("Export Columns:", export_df.columns.tolist())


Import Columns: ['Product', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
Export Columns: ['Product', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


In [11]:
# 📦 Step 1: Import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 📂 Step 2: Load the cleaned export and import datasets
export_clean = pd.read_csv("../data/Processed/Export_Trade_Cleaned.csv")
import_clean = pd.read_csv("../data/Processed/Import_Trade_Cleaned.csv")

# 🧼 Step 3: Clean and reshape the data
def clean_and_melt(df, value_name):
    df = df.copy()
    df.columns = df.columns.str.strip()
    df.iloc[:, 1:] = df.iloc[:, 1:].replace('[\$,]', '', regex=True).astype(float)
    df_long = df.melt(id_vars=["Product"], var_name="Year", value_name=value_name)
    df_long["Year"] = df_long["Year"].astype(int)
    return df_long

export_long = clean_and_melt(export_clean, "Export")
import_long = clean_and_melt(import_clean, "Import")

# 🔗 Step 4: Merge export and import data
merged = pd.merge(export_long, import_long, on=["Product", "Year"])

# 🏅 Step 5: Add export rank and keep top 10 + All Merchandise
merged["Rank"] = merged.groupby("Year")["Export"].rank(method="first", ascending=False)
merged["Is_All_Merchandise"] = merged["Product"].str.contains("All Merchandise")
top_products = merged[(merged["Rank"] <= 10) | (merged["Is_All_Merchandise"])].copy()
top_products["Size"] = top_products["Rank"].apply(lambda r: (11 - r) * 10 if r <= 10 else 120)

# 📊 Step 6: Create animated scatter plot
fig = px.scatter(
    top_products,
    x="Import",
    y="Export",
    color="Product",
    animation_frame="Year",
    size="Size",
    hover_name="Product",
    hover_data={"Rank": True, "Export": True, "Import": True},
    title="U.S. Export vs Import Trade (Top 10 Products + All Merchandise, 2009–2024)"
)

# ➕ Step 7: Add 45-degree trade balance line
max_val = max(top_products["Export"].max(), top_products["Import"].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Trade Balance = 0'
))

# 🧾 Step 8: Final layout adjustments
fig.update_layout(
    xaxis_title="Import Value (Million USD)",
    yaxis_title="Export Value (Million USD)",
    legend_title="Product Category",
    xaxis=dict(range=[0, max_val * 1.05]),
    yaxis=dict(range=[0, max_val * 1.05]),
    height=700
)

fig.show()


<>:14: SyntaxWarning: invalid escape sequence '\$'
<>:14: SyntaxWarning: invalid escape sequence '\$'
/var/folders/4h/1n1m5c3s6vg_drmsgcbr1lz80000gn/T/ipykernel_50120/2650278000.py:14: SyntaxWarning: invalid escape sequence '\$'
  df.iloc[:, 1:] = df.iloc[:, 1:].replace('[\$,]', '', regex=True).astype(float)


In [12]:
# 📦 Step 1: Import libraries
import pandas as pd
import plotly.graph_objects as go

# 📂 Step 2: Load cleaned data
export_clean = pd.read_csv("../data/Processed/Export_Trade_Cleaned.csv")
import_clean = pd.read_csv("../data/Processed/Import_Trade_Cleaned.csv")

# 🧼 Step 3: Clean and reshape data
def clean_and_melt(df, value_name):
    df = df.copy()
    df.columns = df.columns.str.strip()
    df.iloc[:, 1:] = df.iloc[:, 1:].replace('[\$,]', '', regex=True).astype(float)
    df_long = df.melt(id_vars=["Product"], var_name="Year", value_name=value_name)
    df_long["Year"] = df_long["Year"].astype(int)
    return df_long

export_long = clean_and_melt(export_clean, "Export")
import_long = clean_and_melt(import_clean, "Import")

# 🔗 Step 4: Merge & Rank
merged = pd.merge(export_long, import_long, on=["Product", "Year"])
merged["Rank"] = merged.groupby("Year")["Export"].rank(method="first", ascending=False)
merged["Is_All_Merchandise"] = merged["Product"].str.contains("All Merchandise")

# ✂️ Step 5: Split out datasets
all_merch = merged[merged["Is_All_Merchandise"]].copy()
top10 = merged[(~merged["Is_All_Merchandise"]) & (merged["Rank"] <= 10)].copy()
top10["Size"] = (11 - top10["Rank"]) * 10

# 📆 Step 6: Setup animation
years = sorted(top10["Year"].unique())
max_val = max(top10["Export"].max(), top10["Import"].max())

# 🎬 Step 7: Build initial frame
fig = go.Figure()

initial_year = years[0]
initial_top = top10[top10["Year"] == initial_year]
initial_all = all_merch[all_merch["Year"] == initial_year].iloc[0]

# Top 10 traces
for _, row in initial_top.iterrows():
    fig.add_trace(go.Scatter(
        x=[row["Import"]],
        y=[row["Export"]],
        mode='markers',
        name=row["Product"],
        marker=dict(size=row["Size"]),
        hovertemplate=f"<b>{row['Product']}</b><br>Export: {row['Export']:,.0f}<br>Import: {row['Import']:,.0f}<br>Rank: {int(row['Rank'])}<extra></extra>"
    ))

# All Merchandise trace
fig.add_trace(go.Scatter(
    x=[initial_all["Import"]],
    y=[initial_all["Export"]],
    mode='markers+text',
    name="All Merchandise",
    marker=dict(size=14, color='gray', symbol='diamond'),
    text=["All Merchandise"],
    textposition="top center",
    hovertemplate=f"<b>All Merchandise</b><br>Export: {initial_all['Export']:,.0f}<br>Import: {initial_all['Import']:,.0f}<extra></extra>"
))

# Add trade balance line
fig.add_shape(
    type="line",
    x0=0, y0=0, x1=max_val, y1=max_val,
    line=dict(dash="dash", color="black"),
)

# 🎞️ Step 8: Build animation frames
frames = []
for year in years:
    frame_data = []

    year_data = top10[top10["Year"] == year]
    for _, row in year_data.iterrows():
        frame_data.append(go.Scatter(
            x=[row["Import"]],
            y=[row["Export"]],
            mode='markers',
            name=row["Product"],
            marker=dict(size=row["Size"]),
            hovertemplate=f"<b>{row['Product']}</b><br>Export: {row['Export']:,.0f}<br>Import: {row['Import']:,.0f}<br>Rank: {int(row['Rank'])}<extra></extra>"
        ))

    all_row = all_merch[all_merch["Year"] == year].iloc[0]
    frame_data.append(go.Scatter(
        x=[all_row["Import"]],
        y=[all_row["Export"]],
        mode='markers+text',
        name="All Merchandise",
        marker=dict(size=14, color='gray', symbol='diamond'),
        text=["All Merchandise"],
        textposition="top center",
        hovertemplate=f"<b>All Merchandise</b><br>Export: {all_row['Export']:,.0f}<br>Import: {all_row['Import']:,.0f}<extra></extra>"
    ))

    frames.append(go.Frame(data=frame_data, name=str(year)))

# 🧾 Step 9: Final layout and play button
fig.update_layout(
    title="U.S. Export vs Import: Top 10 Products + All Merchandise (Animated, 2009–2024)",
    xaxis=dict(title="Import Value (Million USD)", range=[0, max_val * 1.05]),
    yaxis=dict(title="Export Value (Million USD)", range=[0, max_val * 1.05]),
    height=700,
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(label="Play", method="animate", args=[None, {"frame": {"duration": 800, "redraw": True}, "fromcurrent": True}])]
    )]
)

# 🎞️ Assign frames correctly
fig.frames = frames

# ▶️ Show plot
fig.show()


<>:13: SyntaxWarning:

invalid escape sequence '\$'

<>:13: SyntaxWarning:

invalid escape sequence '\$'

/var/folders/4h/1n1m5c3s6vg_drmsgcbr1lz80000gn/T/ipykernel_50120/2500795844.py:13: SyntaxWarning:

invalid escape sequence '\$'



In [13]:
# 📦 Step 1: Import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# 📂 Step 2: Load data
export_clean = pd.read_csv("../data/Processed/Export_Trade_Cleaned.csv")
import_clean = pd.read_csv("../data/Processed/Import_Trade_Cleaned.csv")

# 🧼 Step 3: Clean and reshape
def clean_and_melt(df, value_name):
    df = df.copy()
    df.columns = df.columns.str.strip()
    
    # Remove $ and commas and convert to float
    for col in df.columns[1:]:
        df[col] = df[col].replace('[\$,]', '', regex=True).astype(float)
    
    # Reshape
    df_long = df.melt(id_vars=["Product"], var_name="Year", value_name=value_name)
    df_long["Year"] = df_long["Year"].astype(int)
    return df_long

export_long = clean_and_melt(export_clean, "Export")
import_long = clean_and_melt(import_clean, "Import")

# 🔗 Step 4: Merge & compute ranks
merged = pd.merge(export_long, import_long, on=["Product", "Year"])
merged["Is_All_Merchandise"] = merged["Product"].str.contains("All Merchandise")

# 🏆 Step 5: Select top 10 export products per year (excluding All Merchandise)
ranked = merged[~merged["Is_All_Merchandise"]].copy()
ranked["Rank"] = ranked.groupby("Year")["Export"].rank(method="first", ascending=False)

# Select top 10 per year
top_products = ranked.groupby("Year").apply(lambda g: g.nsmallest(10, "Rank")).reset_index(drop=True)

# Bubble size based on export value (scale down if needed)
top_products["Size"] = top_products["Export"] / 1000  # You can adjust divisor to scale

# 🧪 Step 6: Animated Scatter Plot
fig = px.scatter(
    top_products,
    x="Import",
    y="Export",
    color="Product",
    animation_frame="Year",
    size="Size",
    hover_name="Product",
    hover_data={"Rank": True, "Export": True, "Import": True},
    title="U.S. Export vs Import Trade (Top 10 Exported Products, 2009–2024)"
)

# ➕ Step 7: Add 45° trade balance line
max_val = max(top_products["Export"].max(), top_products["Import"].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Trade Balance = 0'
))

# 🎨 Step 8: Layout
fig.update_layout(
    xaxis_title="Import Value (Million USD)",
    yaxis_title="Export Value (Million USD)",
    legend_title="Product Category",
    xaxis=dict(range=[0, max_val * 1.05]),
    yaxis=dict(range=[0, max_val * 1.05]),
    height=700
)

fig.show()


<>:17: SyntaxWarning:

invalid escape sequence '\$'

<>:17: SyntaxWarning:

invalid escape sequence '\$'

/var/folders/4h/1n1m5c3s6vg_drmsgcbr1lz80000gn/T/ipykernel_50120/4105148.py:17: SyntaxWarning:

invalid escape sequence '\$'

/var/folders/4h/1n1m5c3s6vg_drmsgcbr1lz80000gn/T/ipykernel_50120/4105148.py:36: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

